# Langchain test

---
## Documentation

> LangChain resources
> - Landpage: https://readthedocs.org/projects/langchain/db2d
> - git: https://github.com/hwchase17/langchain.git
> - API Reference: https://api.python.langchain.com/en/latest/

> Tutos
> - https://towardsdatascience.com/a-gentle-intro-to-chaining-llms-agents-and-utils-via-langchain-16cd385fca81
> - videos Greg Kamradt on YouTube



---
## Setup

In [2]:
pip install langchain


  Using cached langchain-0.0.215-py3-none-any.whl (1.1 MB)
  Using cached aiohttp-3.8.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.0 MB)
  Using cached async_timeout-4.0.2-py3-none-any.whl (5.8 kB)
  Using cached dataclasses_json-0.5.8-py3-none-any.whl (26 kB)
  Using cached langchainplus_sdk-0.0.17-py3-none-any.whl (25 kB)
  Using cached numexpr-2.8.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (381 kB)
  Using cached openapi_schema_pydantic-1.2.4-py3-none-any.whl (90 kB)
  Using cached pydantic-1.10.9-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
  Using cached tenacity-8.2.2-py3-none-any.whl (24 kB)
  Using cached multidict-6.0.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (114 kB)
  Using cached yarl-1.9.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (268 kB)
  Using cached frozenlist-1.3.3-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (149 kB)
  

In [3]:
pip install openai


  Using cached openai-0.27.8-py3-none-any.whl (73 kB)
Note: you may need to restart the kernel to use updated packages.


---
## API keys and configuration

In [4]:
%%bash --out secrets 
# using AWS's Secret Manager to store keys
# garb the keys and store it into a Pytthon variable
export RESPONSE=$(aws secretsmanager get-secret-value --secret-id 'labbenach/sednara/api_keys' )
export SECRETS=$( echo $RESPONSE | jq '.SecretString | fromjson')

echo $SECRETS

In [5]:
import os

os.environ["OPENAI_API_KEY"] = eval(secrets)["OPENAI_API_KEY"]


---
# 1. Basic features

---
## Get prediction from a langage model

In [79]:
from langchain.llms import OpenAI

# loads the model.
# OPENAI_API_KEY is requested. Get it from the OpenAI site.
# a paid account and available units are requested to be able to place a request.
llm = OpenAI(temperature=0.9)

text = "what are the 5 best countries in Europe"

# Actual API call - may tale a while.
print(llm(text))




1. Germany 
2. Switzerland 
3. Denmark 
4. Norway 
5. Luxembourg


---
## Manage prompts

In [80]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

# loads the model.
llm = OpenAI(temperature=0.9)

# setup a prompt
prompt = PromptTemplate (
    input_variables=["interest"],
    template="what are the 5 best countries in Europe ranked on {interest}"
)

In [81]:
text = prompt.format(interest="food")
print(f"{text=}")
print(llm(text))

text='what are the 5 best countries in Europe ranked on food'


1. Italy 
2. France 
3. Spain 
4. Greece 
5. Portugal


In [82]:
text = prompt.format(interest="siteseeing")
print(f"{text=}")
print(llm(text))

text='what are the 5 best countries in Europe ranked on siteseeing'


1. Italy
2. France
3. Spain
4. Greece
5. United Kingdom


---
## Prompt with multiple tokens 
<div class="alert alert-block alert-warning"> TODO </div>


---
# 2. Chains

<div class="alert alert-block alert-warning"> TODO  what is a chain </div>


---
## Built-in chains

In [83]:
from langchain.chains import PALChain
from langchain.llms import OpenAI

# loads the model.
llm = OpenAI(temperature=0.7)

palchain = PALChain.from_math_prompt(llm=llm, verbose=True)


text = """If my age is half of my dad's age 
and he is going to be 60 next year, 
what is my current age?"""
#palchain.run("If my age is half of my dad's age and he is going to be 60 next year, what is my current age?")
palchain.run(text)




> Entering new  chain...
def solution():
    """If my age is half of my dad's age and he is going to be 60 next year, what is my current age?"""
    dad_age_next_year = 60
    my_age = dad_age_next_year / 2
    result = my_age
    return result

> Finished chain.


'30.0'

<div class="alert alert-block alert-warning"> 
    TODO <br>
    - different result each run <br>
    - and should be 29.5
</div>


> Entering new  chain...
def solution():
    """If my age is half of my dad's age and he is going to be 60 next year, what is my current age?"""
    dad_age_next_year = 60
    my_age_fraction = 0.5
    my_age_now = dad_age_next_year * my_age_fraction
    result = my_age_now
    return result

> Finished chain.
'30.0'

> Entering new  chain...
def solution():
    """If my age is half of my dad's age and he is going to be 60 next year, what is my current age?"""
    dad_age_current = 59
    my_age_current = dad_age_current / 2
    result = my_age_current
    return result

> Finished chain.
'29.5'

---
## Multi-step workflow to feed prompt into the model

In [84]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

# loads the model.
llm = OpenAI(temperature=0.9)

# setup a prompt
prompt = PromptTemplate (
    input_variables=["interest"],
    template="what are the 5 best countries in Europe ranked on {interest}"
)

# chain feeds the prompt into the langage mmodel.
chain = LLMChain(llm=llm, prompt=prompt)

In [85]:
chain.run("science")

'\n\n1. Germany\n2. Sweden\n3. Switzerland\n4. United Kingdom\n5. Netherlands'

In [86]:
print(chain.run("tv shows"))



1. United Kingdom
2. France
3. Germany
4. Italy
5. Spain


---
## Using OpenAI Chat API (less expensive)
requires a chain to feed the prompt into the chat 

Other Chat APIs
- https://api.python.langchain.com/en/latest/modules/chat_models.html

In [87]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

chatopenai = ChatOpenAI(model_name="gpt-3.5-turbo")

prompt = PromptTemplate (
    input_variables=["interest"],
    template="what are the 5 best countries in Europe ranked on {interest}"
)

llmchain_chat = LLMChain(llm=chatopenai, prompt=prompt)
print(llmchain_chat.run("food"))


As an AI language model, I do not have personal preferences. However, the following are five countries in Europe that are known for their delicious cuisine:

1. Italy - Italian cuisine is famous for its pasta, pizza, gelato, and wines. Italy is also known for its flavorful seafood and meat dishes.

2. France - French cuisine is renowned for its delicate flavors and rich sauces. It includes dishes such as coq au vin, ratatouille, and escargots.

3. Spain - Spanish cuisine is known for its tapas, paella, and seafood dishes. It also features delicious cured meats and cheeses.

4. Greece - Greek cuisine is characterized by fresh vegetables, grilled meats, and flavorful dips such as tzatziki and hummus. Greek cuisine also includes dishes like moussaka and souvlaki.

5. Turkey - Turkish cuisine is a fusion of Middle Eastern and Mediterranean flavors. It includes dishes such as kebabs, baklava, and Turkish delight. Turkish cuisine also features delicacies like stuffed grape leaves and Turkish

---
## Leverage LLM Math

Evaluating chains that know how to do math.

https://python.langchain.com/docs/guides/evaluation/llm_math

In [88]:
from langchain.prompts import load_prompt
from langchain.chains import LLMMathChain

# loads the model.
llm = OpenAI(temperature=0.9)

prompt = load_prompt('lc://prompts/llm_math/prompt.json')

# deprecated
##chain = LLMMathChain(llm=llm, prompt=prompt)

chain = LLMChain(llm=llm, prompt=prompt)

print(chain.run("what is the largest prime number lower than 20"))


No `_type` key found, defaulting to `prompt`.


Answer: 19


---
# 3. Tools

<div class="alert alert-block alert-warning"> TODO  what is a tool </div>


---
## Leverage Goocle Search

>How to configure the Google search in Langchain 
> - https://python.langchain.com/docs/ecosystem/integrations/google_search

> Custom Search Engine configuration 
> - https://stackoverflow.com/questions/37083058/programmatically-searching-google-in-python-using-custom-search

> CSE API 
> - repo: https://github.com/google/google-api-python-client
> - more info: https://developers.google.com/api-client-library/python/apis/customsearch/v1
> - complete docs: https://api-python-client-doc.appspot.com/

> Get an API key
> - https://developers.google.com/custom-search/v1/introduction

In [89]:
pip install google-api-python-client

  Using cached google_api_python_client-2.90.0-py2.py3-none-any.whl (11.4 MB)
  Using cached httplib2-0.22.0-py3-none-any.whl (96 kB)
  Using cached google_auth-2.20.0-py2.py3-none-any.whl (181 kB)
  Using cached google_auth_httplib2-0.1.0-py2.py3-none-any.whl (9.3 kB)
  Using cached google_api_core-2.11.1-py3-none-any.whl (120 kB)
  Using cached uritemplate-4.1.1-py2.py3-none-any.whl (10 kB)
  Using cached googleapis_common_protos-1.59.1-py2.py3-none-any.whl (224 kB)
  Using cached cachetools-5.3.1-py3-none-any.whl (9.3 kB)
  Using cached urllib3-1.26.16-py2.py3-none-any.whl (143 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.3
    Uninstalling urllib3-2.0.3:
      Successfully uninstalled urllib3-2.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.3.2

In [90]:
# Unlock the API and get a key 
os.environ["GOOGLE_API_KEY"] = eval(secrets)["GOOGLE_API_KEY"]
# Create or use an existing Custom Search Engine
# on the CSE page under Searcg Engone ID
os.environ["GOOGLE_CSE_ID"] = eval(secrets)["GOOGLE_CSE_ID"]


In [91]:
from langchain.tools import Tool
from langchain.utilities import GoogleSearchAPIWrapper

search = GoogleSearchAPIWrapper()

tool = Tool(
    name="Google Search",
    description="Search Google for recent results.",
    func=search.run,
)

tool.run("French Prime Minister name?")

"Élisabeth Borne has served as Prime Minister since 16 May 2022. Fifth Republic recordsEdit. Length of the successive governments\xa0... May 16, 2022 ... President Emmanuel Macron has named Labour Minister Elisabeth Borne as prime minister to lead his ambitious reform plans, the first woman to\xa0... Feb 22, 2018 ... SEVEN months after their prime minister was appointed in May 2017, fully 35% of the French could not name him accurately in a poll. May 16, 2022 ... Elisabeth Borne has been named the new Prime Minister of France, the first time in 30 years that a woman has held the position. May 16, 2022 ... Élisabeth Borne, the minister of labor who previously was in charge of the environment, will be the second woman to hold the post in France. Jun 24, 2022 ... The name of Lafayette is famous and respected on both sides of the Atlantic. It is our third conversation in a month, which is quite a good\xa0... May 5, 2017 ... France's Macron says he has chosen prime minister, won't reveal na

---
# 4. Agent

<div class="alert alert-block alert-warning"> TODO  what is an agent </div>


---
## Setup an agent

In [105]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI

# create a model
llm = OpenAI(temperature=0)

# load some tools
tools = load_tools(["google-search", "llm-math"], llm=llm)

# setup an agent
agent = initialize_agent(tools, 
                         llm, 
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
                         verbose=True)


In [106]:
agent.run("How many Teslas have been sold in 2022. Multiple by 2")



> Entering new  chain...
 I need to find out how many Teslas have been sold in 2022
Action: google_search
Action Input: "how many Teslas have been sold in 2022"
Observation: Apr 15, 2023 ... Tesla total revenue for 2022 was 81,462 billion USD. We show it from 2018 – 2022. Tesla annual revenue 2018 - 2022. Year, Annual ... Jun 7, 2023 ... How many Tesla vehicles were delivered in 2023? ... As of June 2022, Tesla was the most valuable brand within the global automotive sector. Jan 25, 2023 ... The Model 3 and Model Y make up around 95% of the 1.31 million Teslas sold in 2022. Tesla. Tesla's finished 2022 on a tear, bolstered by recent ... Jan 7, 2023 ... Overall, Tesla reported delivering about 1.25 million Model Y and Model 3 vehicles globally in 2022. The Model 3 ranked 13th in sales at 211,641 ... Jan 3, 2023 ... The electric automaker delivered 1.3 million vehicles in 2022, up 40% from 2021. It produced nearly 1.4 million vehicles, up 47% from the prior ... May 30, 2022 ... If ever

'2,620,000 Teslas were sold in 2022.'

In [107]:
agent.run("""Who is the current prime minister of France. 
Is he or sheyounger than the President?""") 



> Entering new  chain...
 I need to find out who the current prime minister is and then compare their age to the President.
Action: google_search
Action Input: "current prime minister of France"
Observation: PresentEdit. Élisabeth Borne has served as Prime Minister since 16 May 2022. Fifth Republic records ... May 16, 2022 ... Who is France's new Prime Minister Elisabeth Borne? French President Emmanuel Macron picked Labour Minister Elisabeth Borne as his new prime ... The current Prime Minister of France is Élisabeth Borne. She was given the job by President Emmanuel Macron on 16 May 2022. May 16, 2022 ... President Emmanuel Macron has named Labour Minister Elisabeth Borne as prime minister to lead his ambitious reform plans, the first woman to ... May 16, 2022 ... Élisabeth Borne, the minister of labor who previously was in charge of the environment, will be the second woman to hold the post in France. May 2, 2014 ... On the recommendation of the Prime Minister, President Hollande 

'The current Prime Minister of France, Élisabeth Borne, is younger than the President, Emmanuel Macron, who is 39 years old.'

In [108]:
agent.run("""Who is the current prime minister of France. 
When will he or she be 70?""") 



> Entering new  chain...
 I need to find out who the current prime minister is and when they will be 70.
Action: google_search
Action Input: "current prime minister of France"
Observation: PresentEdit. Élisabeth Borne has served as Prime Minister since 16 May 2022. Fifth Republic records ... May 16, 2022 ... Who is France's new Prime Minister Elisabeth Borne? French President Emmanuel Macron picked Labour Minister Elisabeth Borne as his new prime ... The current Prime Minister of France is Élisabeth Borne. She was given the job by President Emmanuel Macron on 16 May 2022. May 16, 2022 ... President Emmanuel Macron has named Labour Minister Elisabeth Borne as prime minister to lead his ambitious reform plans, the first woman to ... May 16, 2022 ... Élisabeth Borne, the minister of labor who previously was in charge of the environment, will be the second woman to hold the post in France. May 2, 2014 ... On the recommendation of the Prime Minister, President Hollande has appointed the f

'Élisabeth Borne will be 70 in the year 2092.'

---
# 5. Memory - Conversation

<div class="alert alert-block alert-warning"> TODO  what is a conversation </div>


In [98]:
from langchain import OpenAI, ConversationChain

# create a model
llm = OpenAI(temperature=0)

conversation = ConversationChain(llm=llm, verbose=True)

conversation.predict(input="Hi There")





> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi There
AI:

> Finished chain.


" Hi there! It's nice to meet you. How can I help you today?"

In [99]:
conversation.predict(input="What is the first thing that I said to you?")




> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi There
AI:  Hi there! It's nice to meet you. How can I help you today?
Human: What is the first thing that I said to you?
AI:

> Finished chain.


' You said "Hi there!"'

In [100]:
conversation.predict(input="What is an alternative for the first thing that I said to you?")




> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi There
AI:  Hi there! It's nice to meet you. How can I help you today?
Human: What is the first thing that I said to you?
AI:  You said "Hi there!"
Human: What is an alternative for the first thing that I said to you?
AI:

> Finished chain.


' An alternative for the first thing you said to me is "Hello!"'

---
# 6. Schemas

There are 3 types of schemas
- text (see above)
- Messages 
- Document

---
## Text

---
## Chat messages
Chat messages are like text with a type

There are 3 types
- System: background context that tells the AI what to do
- Human: inputs sent by the user
- AI : response of the AI


In [30]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

chat = ChatOpenAI(temperature=0.7)

In [31]:
messages = [ SystemMessage(content="You are a nice AI and help users to feature out what to eat.")]
     
messages.append( HumanMessage(content="I like tuna, list some recipes.") )

In [32]:
response = chat(messages)
messages.append( AIMessage(content=response.content) )

print(response.content)

Sure, here are some tuna recipes that you might enjoy:

1. Tuna salad: Mix canned tuna with chopped celery, onions, and mayonnaise. You can also add some chopped pickles, mustard, and salt and pepper to taste. Serve on a bed of lettuce or between two slices of bread.

2. Tuna melt: Spread canned tuna on a slice of bread, top with sliced tomato and cheese, and broil until the cheese is melted and bubbly.

3. Tuna pasta salad: Combine cooked pasta with canned tuna, chopped vegetables like bell peppers and onions, and a dressing made of mayonnaise, lemon juice, and herbs.

4. Tuna patties: Mix canned tuna with bread crumbs, egg, and seasonings like garlic, onion powder, and parsley. Form into patties and pan-fry until golden brown.

5. Tuna poke bowl: Top cooked rice with cubed raw tuna, avocado, cucumber, and edamame. Drizzle with a soy sauce-based dressing and garnish with sesame seeds.

I hope these ideas help! Let me know if you have any specific dietary restrictions or preferences, a

In [33]:
messages.append( HumanMessage(content="show the first one.") )

response = chat(messages)
messages.append( AIMessage(content=response.content) )

print(response.content)

Sure, here is a recipe for tuna salad:

Ingredients:
- 2 cans of tuna, drained
- 2 stalks of celery, chopped
- 1 small onion, chopped
- 1/4 cup mayonnaise
- 1 tablespoon chopped pickles (optional)
- 1 teaspoon mustard (optional)
- Salt and pepper to taste

Instructions:
1. In a mixing bowl, combine the drained tuna, chopped celery, and chopped onion.
2. Add the mayonnaise, pickles, and mustard (if using) to the bowl and mix well until everything is combined.
3. Season with salt and pepper to taste.
4. Serve the tuna salad on a bed of lettuce or between two slices of bread.

Enjoy!


---
## Documents

An object that conaints a pieces of text and metadatas

<div class="alert alert-block alert-warning"> TODO how to use this concept? 
make some knowledge available?
how to use metadata?
</div>


In [36]:
from langchain.schema import Document

Document(
    page_content="This is my document. it contains useful information",
    metadata={
        'author':"Claude",
        'identifier':"1234"
    }
)

Document(page_content='This is my document. it contains useful information', metadata={'author': 'Claude', 'identifier': '1234'})

---
# 7. Models

List of models: https://platform.openai.com/docs/models


---
## Langage Model 
Text in Text out 

In [12]:
from langchain.llms import OpenAI

# additnal parameters to select a mode, pass the API key ...
llm = OpenAI(model_name="text-ada-001", temperature=0.7)

llm("What day comes after Friday?")

'\n\nSaturday.'

---
## Chat Model 
Takes a series of messages and return an AI response

Also make sense for a unique interaction as Chat API is less expensive.


In [13]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=1)

In [15]:
messages = [ 
    SystemMessage(content="You are a nice AI and help users to feature out what to eat."),
    HumanMessage(content="I like tuna, list some recipes.")
]
     
chat(messages)

AIMessage(content='Sure, here are some delicious tuna recipes you can try: \n\n1. Tuna Salad: Mix canned tuna with mayonnaise, pickle relish, chopped celery and onion. Serve on salad greens, in a sandwich or on crackers for a light lunch.\n\n2. Tuna Nicoise Salad: Top a bed of salad greens with cooked green beans, boiled potatoes, hard-boiled eggs, canned tuna, and cherry tomatoes. Toss with a simple vinaigrette for a healthy Mediterranean-inspired meal.\n\n3. Tuna Melt: Arrange tuna salad on a slice of crusty bread, top with sliced tomato and cheese, and broil until melted and bubbly.\n\n4. Tuna Poke Bowl: Combine cubed raw tuna with soy sauce, sesame oil, lime juice, green onions, and sesame seeds. Serve over steamed rice with sliced avocado and edamame.\n\n5. Grilled Tuna Steaks: Brush fresh tuna steaks with olive oil and season with salt and pepper. Grill for a few minutes on each side until cooked to your liking, and serve with a side of sautéed vegetables.\n\nI hope these ideas i

---
### Text Embedding Model

Convert text into a series of numbers (a vector) which holds the meaning of the text.

Mainly used for text comparison.

In [19]:
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

text="A leader should know all about truth and honesty, and when to see the difference. (Truck) - Bromeliad Trilogy"

text_embedding = embeddings.embed_query(text)

print(f"embedding length: {len(text_embedding)}")
print(f"5 first values of the vector: {text_embedding[:5]}")

embedding length: 1536
5 first values of the vector: [-0.0020272971596568823, -0.016961609944701195, 0.013975410722196102, -0.014824817888438702, 0.001639920868910849]


---
# 8. prompts

<div class="alert alert-block alert-warning"> TODO </div>